# Implementation for the qFLO Algorithm

### James D. Watson

# Table of contents
1. [Introduction](#introduction)
2. [Methods](#methods)
    1. [Linear Algebra Functions](#linalg)
    2. [Simulation Functions](#simulation)
    3. [Local Observables](#observable)
3. [Hamiltonian Input from OpenFermion](#openfermion)
4. [qDRIFT Implementation](#qDRIFT)

# Introduction <a name="introduction"></a>

Here we give explicit code for the qFLO algorithm given in https://arxiv.org/abs/2411.04240

# Methods <a name="methods"></a>

## Useful Pauli and Qubit Manipulations

In [1]:
# Imports

import numpy as np
import scipy.interpolate as interp
import scipy.linalg as sla
import numpy.linalg as nla
import matplotlib.pyplot as plt
import random as rand
import pandas as pd
from mpmath import *
from datetime import datetime
from datetime import date

C:\Users\05wat\anaconda3\Lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Computes Kronecker (tensor) product of a list of matrices
# Example:
# Input: [X,Y,Z,Y]
# Return: X \ox Y \ox Z \ox Y
def kron_list(matrix_list):
    result = matrix_list[0]
    for i in range(1,len(matrix_list)):
        result = np.kron(result,matrix_list[i])
        
    return result

# Pauli matrices
I = np.array([[1.,0],[0,1.]], dtype='complex')
X = np.array([[0,1.],[1.,0]], dtype='complex')
Y = np.array([[0,-1.j],[1.j,0]], dtype='complex')
Z = np.array([[1.,0],[0,-1.]], dtype='complex')

# Converts string representation of paulis to list of matrices,
# Example:
# Input: "XYZY"
# Return: [X,Y,Z,Y]
def paulistring_to_list(paulistring): 
    matrix_list = list(paulistring)
    translate = {'I':I, 'X':X, 'Y':Y, 'Z':Z}
    for p in range(len(paulistring)):
        matrix_list[p] = translate[matrix_list[p]]
    return matrix_list

# Computes generalized pauli matrix, given a string in standard form,
# Example:
# Input: "XYZY"
# Return: X \ox Y \ox Z \ox Y
def pauli_matrix(paulistring):
    return kron_list(paulistring_to_list(paulistring))

# Computes generalized pauli matrix, specified by non-identity pieces. Nonidentities encoded as dictionary
# of the form {k:'P', ..., } where k is the integer location and P is a pauli
# Example:
# Input: {0: 'X', 3:'Z', 5:'X'}, 6
# Return: X\ox I \ox  Z \ox I \ox I \ox X
def sparse_pauli(nonidentities, nqubits):
    #starting string is all identity
    paulilist = []
    for i in range(0, nqubits):
        paulilist.append('I')
    
    #Change string to paulis specified by dictionary
    for key in nonidentities:
        paulilist[key] = nonidentities[key]
    paulistring = ''.join(paulilist)
    
    return pauli_matrix(paulistring) 

# Returns the matrix of \sigma . \sigma between sites i and j
# Example:
# Input: 0,1,4
# Return: X\ox X \ox I \ox I + Y\ox Y \ox I \ox I + Z\ox Z \ox I \ox I
def sigma_dot_sigma(i,j,nqubits):
    return sparse_pauli({i:'X',j:'X'},nqubits) + sparse_pauli({i:'Y',j:'Y'},nqubits) + sparse_pauli({i:'Z',j:'Z'},nqubits)

# Qubit computational basis
zero = np.array([1.,0], dtype ='complex')
one = np.array([0,1.], dtype ='complex')

# Converts a bitstring to list of single-qubit kets
# Example: 
# Input: "0101"
# Output: list( (1,0), (0,1), (1,0), (0,1) )
def bitstring_to_list(bitstring):
    bitlist = list(bitstring)
    translate = {'0':zero, '1':one}
    for b in range(len(bitstring)):
        bitlist[b] = translate[bitlist[b]]
    return bitlist

# Converts a bitstring to full vector/array describing each qubit.
# Example: 
# Input: "0101"
# Output:  (1,0) \ox (0,1) \ox (1,0) \ox (0,1) 
def basis_ket(bitstring):
    return kron_list(bitstring_to_list(bitstring))


# Outputs a bit string with certain positions set to 1, and otherwise set to zero.
# Example: 
# Input: [0,1,2],3
# Output:  (0,0,0,0,0,0,0,0,1)
def sparse_bitstring(ones, nqubits):
    bitlist = ['0']*nqubits
    
    #Flip certain bits to one, as specified by list of qubit indices
    for qubit in ones:
        bitlist[qubit] = '1'
    
    # Convert list to bitstring
    bitstring = ''.join(bitlist)
    
    return basis_ket(bitstring)

## Linear Algebra Functions <a name="linalg"></a>

In [4]:
## We are working with large matrix powers. Here we use diagonalisation rather than the repeated square method to compute
## large exponents.
## Code taken from: https://saturncloud.io/blog/numpys-matrixpower-function-understanding-and-addressing-incorrect-results-for-large-exponents/

# Computer n^th power of square, diagonalisable matrix A:
def matrix_power_eigenvalue(A, n):
    #Adjust precision
    #mp.dps = 20
    
    ## Use in-built hi-precision function:
    output =  np.array(mp.powm(A, n).tolist(), dtype=complex)
    
    return output

def matrix_power_eigenvalue_precise(A, n, precision=20):
    #Adjust precision
    mp.dps = precision
    
    
    ## Use in-built hi-precision function:
    #output =  np.array(mp.powm(A, n).tolist(), dtype=complex)
    output =  mp.powm(A, n)
   
    return output

# The old function
def matrix_power_eigenvalue_old(A, n):
   
    
    eigenvalues, eigenvectors = np.linalg.eig(A)
    D = np.diag(eigenvalues**n)
    print(D)
    output = fp.matrix( eigenvectors @ D @ np.linalg.inv(eigenvectors))
    
    
    return output

### Simulation Functions <a name="simulation"></a>

In [5]:
# Given: a positive integer, nqubits, and a list of n floats, hlist
# Generates the 1D Heisenberg chain Hamiltonian specified in the introduction.
def heisenbergH(nqubits, hlist):
    H = sigma_dot_sigma(0,1,nqubits) + hlist[0]*sparse_pauli({0:'Z'}, nqubits)
    for j in range(1, nqubits):
        H += sigma_dot_sigma(j, (j+1)%nqubits, nqubits) + hlist[j]*sparse_pauli({j:'Z'}, nqubits)
    return H

# Given a square matrix H and float T
# Returns the exact time evolution operator e^{-i H T} 
def Uexact(H, T):
    return sla.expm(-1.j*H*T)

## Returns the matrix with high precision exponentiation.
def Uexact_precise(H, T):
    
    #Adjust precision
    mp.dps = 20
    
    Uexact = mp.expm( -1.j*H*T , method='taylor')
    
    
    ## Use in-built hi-precision function:
    output =  np.array(Uexact.tolist(), dtype=complex)
    
    return output


# Computes a single time step of first or second order Trotter
# Terms in Hterms are applied to the state in increasing index (0,1,...,m)
def Utrot_short(Hterms, t, order = 1):
    m = len(Hterms) # number of terms
    result = sla.expm(-1.j*Hterms[m-1]*t)
    if order == 1:  
        for k in range(m-2,-1,-1):
            result = result @ sla.expm(-1.j*Hterms[k]*t)
    
    elif order == 2: 
        for k in range(m-2, -1, -1):
            result = sla.expm(-1.j*Hterms[k]*t/2) @ result @ sla.expm(-1.j*Hterms[k]*t/2)
    else:
        raise ValueError("Not a valid order of Trotter formula: must be 1 or 2")
    return result

# Computes a single time step of first or second order Trotter
# Terms in Hterms are applied to the state in increasing index (0,1,...,m)
def Utrot_short_precise(Hterms, t, order = 1, precision =20):
    
    #Adjust precision
    mp.dps = precision
    
    m = len(Hterms) # number of terms
    result = mp.expm(-1.j*Hterms[m-1]*t)
    if order == 1:  
        for k in range(m-2,-1,-1):
            result = result @ mp.expm(-1.j*Hterms[k]*t)
    
    elif order == 2: 
        for k in range(m-2, -1, -1):
            result = mp.expm(-1.j*Hterms[k]*t/2) @ result @ mp.expm(-1.j*Hterms[k]*t/2)
    else:
        raise ValueError("Not a valid order of Trotter formula: must be 1 or 2")
    return result
    
# Given: a list Hterms of square matrices of same dimension, float T, a (possibly noninteger) number of steps, and order =1,2
# Returns a Trotter evolution of given order and step number, for time T
def Utrot_long(Hterms, T, steps, order = 1):
    single_step = Utrot_short(Hterms, T/steps, order)
    
    integer_steps = int( np.floor(steps) )
    fractional_step = steps - integer_steps
    
    integer_step_evolve = matrix_power_eigenvalue(single_step, integer_steps)
    fractional_step_evolve = sla.fractional_matrix_power(single_step, fractional_step)
    
    total_evolution = integer_step_evolve @ fractional_step_evolve 
    
    return total_evolution

## Above, but high precision
def Utrot_long_precise(Hterms, T, steps, order = 1, precision =20):
    
    #Adjust precision
    mp.dps = precision
    
    
    single_step = Utrot_short_precise(Hterms, T/steps, order)
    
    step_evolve = matrix_power_eigenvalue(single_step, steps)
    #fractional_step_evolve = sla.fractional_matrix_power(single_step, fractional_step)
    
    total_evolution = step_evolve 
    
    return total_evolution

# Local Observable and Expectation Value Functions <a name="observable"></a>

In [65]:
## Define a Local Observable:

## Computes  Hermitian observable, specified by Pauli strings. Input is a list of dictionaries: nonidentities encoded as dictionary
## of the form {k:'P', ..., } where k is the integer location and P is a pauli. Pauli strings are then input as list.
# Example:
# Input: [{0: 'X', 3:'Z', 5:'X'}, {1: 'X'}], 6
# Return: X\ox I \ox  Z \ox I \ox I \ox X + I\ox X \ox  I \ox I \ox I \ox I
def intialise_observable(List_Pauli_strings, nqubits):

    local_observable = 0
    for k in range(len(List_Pauli_strings)):
        local_observable +=  sparse_pauli( List_Pauli_strings[k], nqubits )
    
    return local_observable

## Take expectation value of 'observable' with respect to to 'state'
# Example:
# Input: sparse_bitstring([0], 1), observable([{0: 'Z'}], 1) 
# Output: -1
def expectation_value(state, observable):
    
    normalisation =   np.vdot(state, state )  
    # Throw an exception if the state is not normalised.
    if (normalisation -1) > 0.00001:
        raise ValueError("State is not normalised")
    
    # Calculate the expectation of the state. Use vdot to get complex conjugate.
    expectation = np.vdot(state, observable.dot(state) )
    
    # Throw an exception if there is a significant imaginary part to the expectation value.
    if np.abs(expectation.imag) > 0.00001:
        raise ValueError("Expectation value should be entirely real")
        
    # Return only the real part of the expectation value (imaginary part should be zero).
    return np.real(expectation)

## Finds the expectation value with respect to the Trotterized time evolution. Define the function in terms of s = 1/steps.
# Example: 
# Input: [Even, Odd, Potential], sparse_bitstring([0,1], 2), 10, 0.01, observable([{0: 'Z'}], 2) 
# Output: -1 + 0.j
def Trotterized_Expectation(Hterms, initial_state, time, s, observable, order = 1):
    
    steps = 1/s
    
    # Calculate the Trotterized time-evolution operator and apply to the initial state.
    Trotterized_evolution_operator = Utrot_long(Hterms, time, steps, order)
    time_evolved_state = Trotterized_evolution_operator @ initial_state
    
    return expectation_value(time_evolved_state, observable)

## High Precision Version of above.
def Trotterized_Expectation_precise(Hterms, initial_state, time, s, observable, order = 1):
    
    steps = 1/s
    
    # Calculate the Trotterized time-evolution operator and apply to the initial state.
    Trotterized_evolution_operator = Utrot_long_precise(Hterms, time, steps, order)
    time_evolved_state = Trotterized_evolution_operator @ initial_state
    
    return expectation_value(time_evolved_state, observable)
    
## Computes the time-evolved expectation value from the *exact* evolution.  
# Example: 
# Input: heisenbergH(nqubits, hlist), sparse_bitstring([0,1], 2), 10, observable([{0: 'Z'}], 2) 
# Output: -1 + 0.j
def Exact_Expectation(Hamiltonian, initial_state, time, observable):
    
    # Calculate the exact evolution operator.
    evolution_operator = Uexact(Hamiltonian, time)
    # Apply the exact evolution to the initial state.
    time_evolved_state = evolution_operator @ initial_state
    
    return expectation_value(time_evolved_state, observable)


## Precise version of the above.
def Exact_Expectation_Precise(Hamiltonian, initial_state, time, observable):
    
    # Calculate the exact evolution operator.
    evolution_operator = Uexact_precise(Hamiltonian, time)
    # Apply the exact evolution to the initial state.
    time_evolved_state = evolution_operator @ initial_state
    
    return expectation_value(time_evolved_state, observable)


## Computes the magnetisation as a matrix.
# Example:
# Input: 5
# Output: (1/5)\sum_{i=1}^5 Z_i
def Magnetisation(nqubits):
    
    magnetisation_list = []
    
    # For each qubit we want to measure Z on that qubit and then add them together. Generate the Pauli string
    # representing this operator.
    for k in range(nqubits):
        magnetisation_list.append( { k : 'Z'} )
        
    # Calculate the matrix form of the local observable.
    magnetisation_matrix = (1/nqubits)*observable(magnetisation_list, nqubits)
    
    return magnetisation_matrix

# Hamiltonian Input from OpenFermion <a name="openfermion"></a>

In [60]:
### This takes the values stored without going through OpenFermion

#H2
H2_operator = [['-0.16733398905695251', ''], ['-0.046156695889015324', 'X0 X1 Y2 Y3'], ['0.046156695889015324', 'X0 Y1 Y2 X3'], ['0.046156695889015324', 'Y0 X1 X2 Y3'], ['-0.046156695889015324', 'Y0 Y1 X2 X3'], ['0.16251648748871642', 'Z0'], ['0.16583253721590402', 'Z0 Z1'], ['0.11720364720195847', 'Z0 Z2'], ['0.1633603430909738', 'Z0 Z3'], ['0.16251648748871642', 'Z1'], ['0.1633603430909738', 'Z1 Z2'], ['0.11720364720195847', 'Z1 Z3'], ['-0.19744293699755816', 'Z2'], ['0.17169788392286725', 'Z2 Z3'], ['-0.19744293699755816', 'Z3']]

#LiH
LiH_operator = [['-7.49894690201071', ''], ['-0.0029329964409502266', 'X0 X1 Y2 Y3'], ['0.0029329964409502266', 'X0 Y1 Y2 X3'], ['0.01291078027311749', 'X0 Z1 X2'], ['-0.0013743761078958677', 'X0 Z1 X2 Z3'], ['0.011536413200774975', 'X0 X2'], ['0.0029329964409502266', 'Y0 X1 X2 Y3'], ['-0.0029329964409502266', 'Y0 Y1 X2 X3'], ['0.01291078027311749', 'Y0 Z1 Y2'], ['-0.0013743761078958677', 'Y0 Z1 Y2 Z3'], ['0.011536413200774975', 'Y0 Y2'], ['0.16199475388004184', 'Z0'], ['0.011536413200774975', 'Z0 X1 Z2 X3'], ['0.011536413200774975', 'Z0 Y1 Z2 Y3'], ['0.12444770133137588', 'Z0 Z1'], ['0.054130445793298836', 'Z0 Z2'], ['0.05706344223424907', 'Z0 Z3'], ['0.012910780273117487', 'X1 Z2 X3'], ['-0.0013743761078958677', 'X1 X3'], ['0.012910780273117487', 'Y1 Z2 Y3'], ['-0.0013743761078958677', 'Y1 Y3'], ['0.16199475388004186', 'Z1'], ['0.05706344223424907', 'Z1 Z2'], ['0.054130445793298836', 'Z1 Z3'], ['-0.013243698330265966', 'Z2'], ['0.08479609543670981', 'Z2 Z3'], ['-0.013243698330265952', 'Z3']]

In [44]:
## Functions for converting outputs of the Hamiltonian to the 

# Remove the identity terms to get only the non-trivial terms.
def remove_identity(Hamiltonian):
    reduced_Hamiltonian = Hamiltonian
    for element in reduced_Hamiltonian:
        if element[1]=='':
            reduced_Hamiltonian.remove(element)

    return reduced_Hamiltonian
            
# Convert the openfermion operator to our form.
# Input: [['-0.5', 'X0 X1 Y2 Y3'], ['2', 'X0 Y1 Y2 X3']]
# Output: [[-0.5, {0: 'X', 1: 'X', 2: 'Y', 3: 'Y'}], [2, {0: 'X', 1: 'Y', 2: 'Y', 3: 'X'}]]
def openfermion_to_local_rep(of_Hamiltonian):
    local_representation = []
    for element in of_Hamiltonian:
        coefficient     = float(element[0]) 
        element_strings = element[1].split(' ')
        local_operator  = {}
        for op_string in element_strings:
            local_operator[int(op_string[1])]=op_string[0]

        local_representation.append( [ coefficient, local_operator  ] )

    return local_representation

# Transforms into explicit matrix representation.
# Takes input from openfermion_to_local_rep
def local_matrix_representation(local_rep_Hamiltonian, nqubits):
    matrix_representation = []
    for element in local_rep_Hamiltonian:
        matrix_representation.append( [element[0], sparse_pauli(element[1], nqubits)])

    return matrix_representation

# Generates the full explicit matrix of the Hamiltonian.
def explicit_Hamiltonian_matrix(local_matrix_representation):
    Hamiltonian_matrix = 0
    for element in local_matrix_representation:
        Hamiltonian_matrix += element[0]*element[1]

    return Hamiltonian_matrix


In [46]:
## Tests
#test_op = remove_identity(H2_operator)

#new_rep = openfermion_to_local_rep(test_op)

#local_mat_rep = local_matrix_representation(new_rep, 4)

#Hamiltonian_matrix = explicit_Hamiltonian_matrix(local_mat_rep)
#print(len(test_op))
#print(Hamiltonian_matrix)

# qDRIFT Implementation <a name="qDRIFT"></a>

In [80]:
import random

## Takes a list of coefficients and terms and then sums them cummulatively.
def cumulative_Hamiltonian_rep(local_rep_Hamiltonian):
    running_total = 0
    cumulative_representation = []
    
    for element in local_rep_Hamiltonian:
        running_total += np.abs( element[0] )
        cumulative_representation.append( [ running_total,  element[1] ] )

    return cumulative_representation


def binary_search(arr, low, high, x):
 
    # Check base case
    if high >= low:
 
        mid = (high + low) // 2
 
        # If element is present at the middle itself
        if arr[mid] == x:
            return mid
 
        # If element is smaller than mid, then it can only
        # be present in left subarray
        elif arr[mid] > x:
            return binary_search(arr, low, mid - 1, x)
 
        # Else the element can only be present in right subarray
        else:
            return binary_search(arr, mid + 1, high, x)
 
    else:
        # Element is not present in the array
        return "Error, something went wrong"



# Takes the set of samples and picks the corresponding terms.
def terms_to_implement(random_samples, cummulative_rep_Hamiltonian):

    cummulative_array = [x[0] for x in cummulative_rep_Hamiltonian]
    index_list = []
    
    for sample in random_samples:  
        if sample<= cummulative_rep_Hamiltonian[0][0]:
            entry = 0
            index_list.append(entry)
        else:
            #entry = binary_search(cummulative_array, cummulative_array[0], cummulative_array[-1], sample)
            entry = binary_search(cummulative_array, 0, len(cummulative_array)-1, sample)
            index_list.append(entry)
        print("Sample, index:", sample, entry)
    return index_list


def qDRIFT_observable(local_rep_Hamiltonian, initial_state, observable, nqubits, time, epsilon):

    cumulative_Hamiltonian = cumulative_Hamiltonian_rep(local_rep_Hamiltonian)
    lambda_param = cumulative_Hamiltonian[-1][0]

    no_timesteps = np.ceil( 2*time**2*lambda_param**2/epsilon )
    timestep = time/no_timesteps 

    # List of samples.
    samples = np.random.rand(int(no_timesteps))
    #samples = random.sample(range(0, lambda_param), no_timesteps)
    # Now generate the corresponding terms.
    list_of_indicies = terms_to_implement( samples, cumulative_Hamiltonian) 

    # Generate the unitary:
    state = initial_state
    for index in list_of_indicies:
        state =  np.exp(-1.j*lambda_param*timestep*cumulative_Hamiltonian[index]) @state

    # Find the expected value relative to the time-evolved state.
    expect_val = expectation_value(state, observable)

    return expect_val

#### Short Test Case

In [81]:
H2_operator = remove_identity(H2_operator)
H2_local_rep = openfermion_to_local_rep(H2_operator)

nqubits = 4
time = 1
observable = intialise_observable([{0: 'X', 3:'Z'}, {1: 'X'}], nqubits)
state = sparse_bitstring([nqubits//2], nqubits)
epsilon = 0.1


test_val = qDRIFT_observable(H2_local_rep , state, observable, nqubits, time, epsilon )
print(test_val)

Sample, index: 0.7588050946706785 Error, something went wrong
Sample, index: 0.10688902770550401 Error, something went wrong
Sample, index: 0.6935785655661942 Error, something went wrong
Sample, index: 0.6058143491920055 Error, something went wrong
Sample, index: 0.19536359777889545 Error, something went wrong
Sample, index: 0.8259549167805014 Error, something went wrong
Sample, index: 0.45357509520282047 Error, something went wrong
Sample, index: 0.7642137942597225 Error, something went wrong
Sample, index: 0.8415684928401723 Error, something went wrong
Sample, index: 0.36778752960994743 Error, something went wrong
Sample, index: 0.6133608378253019 Error, something went wrong
Sample, index: 0.8917186720079061 Error, something went wrong
Sample, index: 0.7109526920925519 Error, something went wrong
Sample, index: 0.5409949509084105 Error, something went wrong
Sample, index: 0.7969666460522835 Error, something went wrong
Sample, index: 0.7006299616276153 Error, something went wrong
Samp

TypeError: list indices must be integers or slices, not str